# <font face="garamond" size="18" color="#122DAC">***What firm-specific characteristics explain the variation in corporate leverage (debt ratios) among US firms?***</font>
## <font face="garamond" size="6" color="#122DAC">*Econ 430*</font> 
### <font face="garamond" size="6" color="#122DAC">*Jacob Williams, Josh Kentworthy, Ignacio Ramirez, and Tadeh Essakhan*</font>

# University of California Los Angeles
## October, 22 2025


## Research Question: 
## What firm-specific characteristics explain the cross-sectional variation in corporate leverage (debt ratios) among US firms?



## Financial Economic Theories that we are testing (structuring the hypothesis on:

### 1. Trade-Off Theory: Firms balance the tax benefits of debt against the costs of financial distress.

### 2. Pecking Order Theory: Firms prefer internal financing (retained earnings), then debt, then equity as a last resort.


## Data & Variables:


### Data Source: 
#### Balance Sheet info from WRDS - Wharton Research Data Services
####                OR
####             Balance Sheet info from Yahoo Finance
             

## Observations:

### - Dependent Variable (Y): Leverage Ratio (e.g., Total Debt / Total Equity).

### Potential Predictors (X): 

#### Profitability: (e.g., Return on Assets - ROA). Pecking order predicts a negative relationship.

#### Firm Size: (e.g., log(Total Assets)). Trade-off theory predicts a positive relationship (larger firms are more diversified and have lower distress costs).

#### Tangibility: (e.g., Net Property, Plant & Equipment / Total Assets). Trade-off theory predicts a positive relationship (tangible assets can be used as collateral).

#### Growth Opportunities: (e.g., Market-to-Book Ratio). Trade-off theory predicts a negative relationship (high-growth firms have more to lose in distress).

#### Tax Shield: (e.g., Effective Tax Rate or Depreciation Expense / Total Assets).

## <font face="garamond" size="6" color="#122DAC">*Breakdown of variables/paramaters listed above:*</font>

## Corporate Capital Structure: Leverage (Debt Ratio = Debt/Equity) = $LR$


### Profitability: Return on Assets = $ROA$

ROA = (Net Income/Total Assets)

How efficiently a company makes profit from what it owns.

If ROA is high, the firm's assets are productive — it's making good profits without needing much outside money.

Profitable companies usually don't need to borrow as much. They can pay for projects with their own earnings. So, higher ROA - lower leverage.



### Log Firm Size/Asset Size: Log(Total Assets) = $log[TA]$

A measure of how big the company is. We use the log to keep the numbers from blowing up (since asset values can be huge).

Big firms are seen as safer — they're more stable, more diversified, and have better access to credit markets.

So, a larger size usually means higher leverage.


### Tangibility: Net Property, Plant & Equipment / Total Assets = $NP/TA$

Tangibility = (Net Property, plant, and equipment/ Total Assets)

How much of a company's assets are physical things like buildings, machinery, and equipment (as opposed to patents, brand value, or software).

Banks like lending against hard assets — they can use them as collateral if the company defaults.

So, firms with more tangible stuff can safely borrow more.


### Growth Opportunity: Market-to-Book Ratio (Company's current market value to its book value) = $M/B$

M/B = (Market Value of Equity/ Book Value of Equity)

Compares what investors think the firm is worth (market value) vs what the accounting books say it's worth.

If it's high, the market expects lots of growth or future profits.

High-growth firms (like tech) often avoid debt to stay flexible — they don't want to risk bankruptcy before they realize their growth.



### Tax Shield: Effective Tax Rate = $ETR$

ETR = Taxes Paid/(Pre - Tax Income)

How much of a firm's income goes to taxes on average.

Debt gives a tax shield — interest payments are tax-deductible.

So if a firm pays a lot in taxes, using more debt can help reduce that bill.

Higher ETR → more incentive to






$$ LR = \beta_0 = \beta_1[ROA]\beta_2[TA]\beta_3[NP/TA]\beta_4[M/B]\beta_5[ETR] + \epsilon $$

gvkey,    -- Compustat's unique firm identifier

        datadate, -- Date of the financial statements
        
        fyear,    -- Fiscal Year
        
        tic,      -- Ticker symbol
        
        conm,     -- Company Name

        -- --- ASSETS ---
        at,       -- Assets - Total
        act,      -- Current Assets - Total
        che,      -- Cash and Short-Term Investments
        rect,     -- Receivables - Total
        invt,     -- Inventories - Total
        ppent,    -- Property, Plant, and Equipment - Total (Net)

        -- --- LIABILITIES & EQUITY ---
        lt,       -- Liabilities - Total
        lct,      -- Current Liabilities - Total
        dltt,     -- Long-Term Debt - Total
        dlc,      -- Debt in Current Liabilities (Short-Term Debt)
        ceq,      -- Common Equity - Total
        pstk      -- Preferred Stock

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import yfinance as yf
import requests
import wrds

In [7]:
db = wrds.Connection()

Enter your WRDS username [rokinjacob]: jacobw01
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [10]:
# 1. Establish connection to WRDS
# You'll be prompted for your username and password the first time.
print("Connecting to WRDS...")
db = wrds.Connection()
print("Successfully connected to WRDS.")

# --- 2. Set Query Parameters ---

# The ticker symbol you want to query
company_ticker = 'AAPL' 

# The fiscal year you want
fiscal_year = 2024

# --- 3. Write the SQL Query ---
# We are querying the Compustat Fundamentals Annual table: comp.funda
# We select common identifiers and key balance sheet items.

sql_query = f"""
    SELECT 
        gvkey,    -- Compustat's unique firm identifier
        datadate, -- Date of the financial statements
        fyear,    -- Fiscal Year
        tic,      -- Ticker symbol
        conm,     -- Company Name

        -- --- ASSETS ---
        at,       -- Assets - Total
        act,      -- Current Assets - Total
        che,      -- Cash and Short-Term Investments
        rect,     -- Receivables - Total
        invt,     -- Inventories - Total
        ppent,    -- Property, Plant, and Equipment - Total (Net)

        -- --- LIABILITIES & EQUITY ---
        lt,       -- Liabilities - Total
        lct,      -- Current Liabilities - Total
        dltt,     -- Long-Term Debt - Total
        dlc,      -- Debt in Current Liabilities (Short-Term Debt)
        ceq,      -- Common Equity - Total
        pstk      -- Preferred Stock

    FROM 
        comp.funda
    WHERE
        tic = '{company_ticker}'
        AND fyear = {fiscal_year}
        AND datafmt = 'STD'  -- Selects standardized data
        AND indfmt = 'INDL'  -- Selects industrial format
        AND consol = 'C'     -- Selects consolidated statements
        AND popsrc = 'D'     -- Selects domestic data
"""

# --- 4. Execute the Query ---
print(f"Downloading balance sheet for {company_ticker} for fiscal year {fiscal_year}...")
try:
    df = db.raw_sql(sql_query)
    
    print("Download Complete.")
    print("\n--- Balance Sheet Data ---")
    
    # .T (transpose) makes the single row of data easier to read
    print(df.T)

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # 5. Close the connection
    db.close()
    print("WRDS connection closed.")

Connecting to WRDS...


Enter your WRDS username [rokinjacob]: jacobw01
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Successfully connected to WRDS.
Download Complete.

--- Balance Sheet Data ---
                   0
gvkey         001690
datadate  2024-09-30
fyear           2024
tic             AAPL
conm       APPLE INC
at          364980.0
act         152987.0
che          65171.0
rect         66243.0
invt          7286.0
ppent        55914.0
lt          308030.0
lct         176392.0
dltt         96548.0
dlc          22511.0
ceq          56950.0
pstk             0.0
WRDS connection closed.


In [14]:
# --- 1. Set Query Parameters ---
company_ticker = 'AAPL'  # Replace with your ticker
fiscal_year_q = 2025     # Set the fiscal year
fiscal_quarter = 2       # Set the fiscal quarter (Q2)

# --- 2. Establish Connection ---
print("Connecting to WRDS...")
db = wrds.Connection()
print("Successfully connected to WRDS.")

# --- 3. Write the Quarterly SQL Query ---
# Note we are querying 'comp.fundq'
# Note all mnemonics end in 'q'

sql_query = f"""
    SELECT 
        gvkey,    -- Firm identifier
        datadate, -- Date of the quarterly report
        fyearq,   -- Fiscal Year (Quarterly)
        fqtr,     -- Fiscal Quarter (1-4)
        tic,      -- Ticker symbol
        conm,     -- Company Name

        -- --- ASSETS ---
        atq,      -- Assets - Total - Quarterly
        actq,     -- Current Assets - Total - Quarterly
        cheq,     -- Cash - Quarterly
        rectq,    -- Receivables - Total - Quarterly
        invtq,    -- Inventories - Total - Quarterly
        ppentq,   -- Property, Plant, & Equipment - Total (Net) - Quarterly

        -- --- LIABILITIES & EQUITY ---
        ltq,      -- Liabilities - Total - Quarterly
        lctq,     -- Current Liabilities - Total - Quarterly
        dlttq,    -- Long-Term Debt - Total - Quarterly
        dlcq,     -- Debt in Current Liabilities - Quarterly
        ceqq,     -- Common Equity - Total - Quarterly
        pstkq     -- Preferred Stock - Quarterly

    FROM 
        comp.fundq
    WHERE
        tic = '{company_ticker}'
        AND fyearq = {fiscal_year_q}
        AND fqtr = {fiscal_quarter}
        AND datafmt = 'STD'
        AND indfmt = 'INDL'
        AND consol = 'C'
        AND popsrc = 'D'
"""

# --- 4. Execute the Query ---
print(f"Downloading Q{fiscal_quarter} {fiscal_year_q} balance sheet for {company_ticker}...")
try:
    df = db.raw_sql(sql_query)
    
    print("Download Complete.")
    
    if df.empty:
        print("No data found for this ticker and period (it may not be reported yet).")
    else:
        print("\n--- Quarterly Balance Sheet Data ---")
        # .T (transpose) makes the single row of data easier to read
        print(df.T)

except Exception as e:
    print(f"An error occurred: {e}")

Connecting to WRDS...


Enter your WRDS username [rokinjacob]: jacobw01
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Successfully connected to WRDS.
Download Complete.

--- Quarterly Balance Sheet Data ---
                   0
gvkey         001690
datadate  2025-03-31
fyearq          2025
fqtr               2
tic             AAPL
conm       APPLE INC
atq         331233.0
actq        118674.0
cheq         48498.0
rectq        49798.0
invtq         6269.0
ppentq       46876.0
ltq         264437.0
lctq        144571.0
dlttq        78566.0
dlcq         19620.0
ceqq         66796.0
pstkq            0.0
WRDS connection closed.


In [16]:
# --- 1. Establish Connection ---
print("Connecting to WRDS...")
# This will prompt for your username and password
db = wrds.Connection()
print("Successfully connected to WRDS.")

# --- 2. Define the SQL Query ---
# This query is more complex. It joins the S&P 500 constituent list
# with the quarterly balance sheet data for a specific calendar date range.

sql_query = """
    -- CTE to get the list of S&P 500 gvkeys active during Q2 2025
    WITH sp500_gvkeys AS (
        SELECT gvkey
        FROM comp.idx_constituent
        WHERE
            gvkeyx = '000003' -- '000003' is the index gvkey for S&P 500 Composite
            AND "from" <= '2025-06-30' -- Joined on or before the end of Q2 2025
            AND (thru IS NULL OR thru >= '2025-06-30') -- Still in the index at the end of Q2 2025
    )
    
    -- Main query to select balance sheet data
    SELECT 
        q.gvkey,
        q.datadate, -- The specific date of the balance sheet
        q.fyearq,   -- Fiscal Year (Quarterly)
        q.fqtr,     -- Fiscal Quarter (1-4)
        q.tic,      -- Ticker
        q.conm,     -- Company Name
        q.rdq,      -- Report Date (when it was filed)
        
        -- Key Balance Sheet Mnemonics (Quarterly)
        q.atq,      -- Assets - Total
        q.actq,     -- Current Assets - Total
        q.cheq,     -- Cash
        q.rectq,    -- Receivables
        q.invtq,    -- Inventories
        q.ppentq,   -- Property, Plant, & Equipment (Net)
        q.ltq,      -- Liabilities - Total
        q.lctq,     -- Current Liabilities - Total
        q.dlttq,    -- Long-Term Debt - Total
        q.dlcq,     -- Debt in Current Liabilities (Short-Term)
        q.ceqq,     -- Common Equity - Total
        q.pstkq     -- Preferred Stock

    FROM 
        comp.fundq AS q
        
    -- Join with our S&P 500 list
    INNER JOIN 
        sp500_gvkeys AS sp ON q.gvkey = sp.gvkey
        
    WHERE
        -- Filter for reports with a balance sheet date in Q2 2025
        q.datadate BETWEEN '2025-04-01' AND '2025-06-30'
        
        -- Standard data quality filters
        AND q.datafmt = 'STD'
        AND q.indfmt = 'INDL'
        AND q.consol = 'C'
        AND q.popsrc = 'D'
"""

# --- 3. Execute the Query ---
print("Downloading S&P 500 balance sheet data for Q2 2025...")
try:
    df = db.raw_sql(sql_query)
    
    print("Download Complete.")
    print(f"Successfully retrieved {len(df)} company reports.")
    
    if not df.empty:
        print("\n--- Data Head ---")
        print(df.head())
        
        print("\n--- Data Info ---")
        df.info()
        
        # Save to CSV
        output_file = "sp500_q2_2025_balance_sheets.csv"
        df.to_csv(output_file, index=False)
        print(f"\nData saved to {output_file}")

except Exception as e:
    print(f"An error occurred: {e}")

Connecting to WRDS...


Enter your WRDS username [rokinjacob]: jacobw01
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Successfully connected to WRDS.
An error occurred: (psycopg2.errors.UndefinedTable) relation "comp.idx_constituent" does not exist
LINE 5:         FROM comp.idx_constituent
                     ^

[SQL: 
    -- CTE to get the list of S&P 500 gvkeys active during Q2 2025
    WITH sp500_gvkeys AS (
        SELECT gvkey
        FROM comp.idx_constituent
        WHERE
            gvkeyx = '000003' -- '000003' is the index gvkey for S&P 500 Composite
            AND "from" <= '2025-06-30' -- Joined on or before the end of Q2 2025
            AND (thru IS NULL OR thru >= '2025-06-30') -- Still in the index at the end of Q2 2025
    )
    
    -- Main query to select balance sheet data
    SELECT 
        q.gvkey,
        q.datadate, -- The specific date of the balance sheet
        q.fyearq,   -- Fiscal Year (Quarterly)
        q.fqtr,     -- F

In [17]:
df.head

<bound method NDFrame.head of     gvkey    datadate  fyearq  fqtr   tic       conm       atq      actq  \
0  001690  2025-03-31    2025     2  AAPL  APPLE INC  331233.0  118674.0   

      cheq    rectq   invtq   ppentq       ltq      lctq    dlttq     dlcq  \
0  48498.0  49798.0  6269.0  46876.0  264437.0  144571.0  78566.0  19620.0   

      ceqq  pstkq  
0  66796.0    0.0  >

In [18]:
print(df)

    gvkey    datadate  fyearq  fqtr   tic       conm       atq      actq  \
0  001690  2025-03-31    2025     2  AAPL  APPLE INC  331233.0  118674.0   

      cheq    rectq   invtq   ppentq       ltq      lctq    dlttq     dlcq  \
0  48498.0  49798.0  6269.0  46876.0  264437.0  144571.0  78566.0  19620.0   

      ceqq  pstkq  
0  66796.0    0.0  


In [19]:
yf.download("AAPL MSFT SPY", period="7d", interval="5m").to_csv("intra.csv")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


/var/folders/dr/1jpzryjj14g306qp24mp_m540000gn/T/ipykernel_86199/80483214.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  yf.download("AAPL MSFT SPY", period="7d", interval="5m").to_csv("intra.csv")
[*********************100%***********************]  3 of 3 completed


In [20]:
tic = yf.Ticker("AAPL")
hist = tic.history(period="max", interval="1d")  # or start="1990-01-01", end="2025-10-22"
print(hist.head())
hist.to_csv("AAPL_history.csv")


                               Open      High       Low     Close     Volume  \
Date                                                                           
1980-12-12 00:00:00-05:00  0.098485  0.098913  0.098485  0.098485  469033600   
1980-12-15 00:00:00-05:00  0.093775  0.093775  0.093347  0.093347  175884800   
1980-12-16 00:00:00-05:00  0.086924  0.086924  0.086495  0.086495  105728000   
1980-12-17 00:00:00-05:00  0.088636  0.089064  0.088636  0.088636   86441600   
1980-12-18 00:00:00-05:00  0.091206  0.091634  0.091206  0.091206   73449600   

                           Dividends  Stock Splits  
Date                                                
1980-12-12 00:00:00-05:00        0.0           0.0  
1980-12-15 00:00:00-05:00        0.0           0.0  
1980-12-16 00:00:00-05:00        0.0           0.0  
1980-12-17 00:00:00-05:00        0.0           0.0  
1980-12-18 00:00:00-05:00        0.0           0.0  


In [21]:
t = yf.Ticker("AAPL")

# Fast reference fields
print(t.fast_info)                  # market_cap, last_price, float_shares, etc.

# Financial statements (quarterly & annual)
print(t.financials)                 # Income statement (annual)
print(t.quarterly_financials)       # Income statement (quarterly)
print(t.balance_sheet)
print(t.cashflow)

# Corporate actions
print(t.dividends.tail())
print(t.splits.tail())

# Analyst / calendar
print(t.recommendations.tail())
print(t.calendar)                   # earnings dates, etc.


lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']
                                                      2024-09-30  \
Tax Effect Of Unusual Items                         0.000000e+00   
Tax Rate For Calcs                                  2.410000e-01   
Normalized EBITDA                                   1.346610e+11   
Net Income From Continuing Operation Net Minori...  9.373600e+10   
Reconciled Depreciation                             1.144500e+10   
Reconciled Cost Of Revenue                          2.103520e+11   
EBITDA                                              1.346610e+11   
EBIT                                                1.232160e+11   
Net Interest Income                                 